In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/inputdata/new_data1.json
/kaggle/input/inputdata/arabic_conversation.json
/kaggle/input/inputdata/arabic_conversation_simulation.json
/kaggle/input/inputdata/ChatData.py


In [2]:
!pip install transformers
!pip install rouge
!pip install pytorch-lamb

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import json
import random
from rouge import Rouge 
from pytorch_lamb import Lamb



/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:

def calculate_rouge(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return scores


In [5]:
def train(path, model, optim,epochs):
    chatData = ChatData(path, tokenizer)
    chatData =  DataLoader(chatData, num_workers=2, batch_size=64)
    test = ("<Q>كم تبلغ تكلفة التوصيل<bot>:")
    output_referance = "<Q>كم تبلغ تكلفة التوصيل<bot>:تكلفة التوصيل"
    model.train()

    

    print("training .... ")
    
    for i in tqdm.tqdm(range(epochs)):
        for X, a,Y in chatData:
            X = X.to(device)
            a = a.to(device)
            Y = Y.to(device)
            optim.zero_grad()
            #loss = model(X, attention_mask=a, labels=X).loss
            losses = model(X, attention_mask=a, labels=Y).loss
            loss = losses.mean()
            loss.backward()
            optim.step()
        # Save the model after each epoch
        torch.save(model.state_dict(), f"model_state_epoch_{i}.pt")

        # Evaluation...
        model.eval()
        with torch.no_grad():
            output = infer(test)
            print(output)
            rouge_1_r_sum, rouge_1_p_sum, rouge_1_f_sum = 0, 0, 0
            rouge_2_r_sum, rouge_2_p_sum, rouge_2_f_sum = 0, 0, 0
            rouge_l_r_sum, rouge_l_p_sum, rouge_l_f_sum = 0, 0, 0

            for i in range(10000):
                rouge_scores = calculate_rouge(chatData.X[i], chatData.Y[i])[0]  # assuming this function returns a list similar to your example
                rouge_1_r_sum += rouge_scores['rouge-1']['r']
                rouge_1_p_sum += rouge_scores['rouge-1']['p']
                rouge_1_f_sum += rouge_scores['rouge-1']['f']

                rouge_2_r_sum += rouge_scores['rouge-2']['r']
                rouge_2_p_sum += rouge_scores['rouge-2']['p']
                rouge_2_f_sum += rouge_scores['rouge-2']['f']

                rouge_l_r_sum += rouge_scores['rouge-l']['r']
                rouge_l_p_sum += rouge_scores['rouge-l']['p']
                rouge_l_f_sum += rouge_scores['rouge-l']['f']

            # print averages
            print('Average Rouge-1 scores: r:', rouge_1_r_sum/10000, ', p:', rouge_1_p_sum/10000, ', f:', rouge_1_f_sum/10000)
            print('Average Rouge-2 scores: r:', rouge_2_r_sum/10000, ', p:', rouge_2_p_sum/10000, ', f:', rouge_2_f_sum/10000)
            print('Average Rouge-L scores: r:', rouge_l_r_sum/10000, ', p:', rouge_l_p_sum/10000, ', f:', rouge_l_f_sum/10000)
        model.train()
    model.eval()
def infer(inp):
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output =model.module.generate(X, attention_mask=a,max_new_tokens=50 
                        ,pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id  
    ,no_repeat_ngram_size=5
                           )
    output = tokenizer.decode(output[0])
    return output


In [6]:

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r", encoding="utf-8"))
        self.X = []
        self.Y = []
        tmp = ""
        for i in self.data:
            for j in i['dialog']:
                if j['sender'] == "participant2":
                    self.X.append(j['text'])
                    tmp = j['text']+"<bot>:"
                else:
                    self.Y.append(tmp+j['text'])
                    tmp = ""

        #combined = list(zip(self.X, self.Y))
        #random.shuffle(combined)

        #self.X[:], self.Y[:] = zip(*combined)
        print(len(self.X),len(self.Y))
        self.X = self.X[:10000]
        self.Y = self.Y[:10000]
        print(self.Y[0])
        self.X_encoded = tokenizer(self.X, max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.Y_encoded = tokenizer(self.Y, max_length=40, truncation=True, padding="max_length", return_tensors="pt")

        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

        self.target_ids = self.Y_encoded['input_ids']
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx], self.target_ids[idx]


In [7]:
model_name = "aubmindlab/aragpt2-base"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})

tokenizer.add_tokens(["<bot>:","<prevQ>","<prevA>","<Q>"])
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model = model.to(device)
#optim = Lamb(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.01)

optim = Adam(model.parameters(), lr=1e-3)



Let's use 2 GPUs!


In [8]:
train("/kaggle/input/inputdata/new_data1.json", model, optim,3)

In [9]:
def evaluatetion(input):
    output_referance = "<Q>مرحبا<bot>:أهلاً في مؤسسة اكسبو شوب! ما الذي يمكنني أن أفعله من أجلك؟"
    output = infer(input)
    print(output)
    print(calculate_rouge(output, output_referance))

In [10]:
evaluatetion("<Q>مرحبا<bot>:")
evaluatetion("<prevQ>مرحبا<prevA>أهلاً في مؤسسة اكسبو شوب! ما الذي يمكنني أن أفعله من أجلك؟<Q>"+"كم سعر شاشات التلفار"+"<bot>:")
evaluatetion("<prevQ>"+"كم سعر شاشات التلفار"+"<prevA>"+"سعر شاشة سامسونغ 1200"+"<Q>"+"كم تكلفة التوصيل"+"<bot>:")
#evaluatetion("<prevQ>مرحبا"+"<prevQ>"+"كم سعر شاشات التلفار"+"prevA>أهلاً في مؤسسة اكسبو شوب! ما الذي يمكنني أن أفعله من أجلك؟"+"<prevA>"+"سعر شاشة سامسونغ 1200"+"<Q>"+"كم تكلفة التوصيل"+"<bot>:")

<Q> مرحبا <bot>:  ، ، ، ، ، مرحبا ، ، ، مرحبا بك أخي الزائر الكريم. يمكنك الاستفادة من محتوى البرامج على وجه الخصوص و يشرفنا تسجيلك في المنتدى : اضغط على الصورةموضوع : رد : : : : :..... : : : :
[{'rouge-1': {'r': 0.16666666666666666, 'p': 0.08, 'f': 0.10810810372534715}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.08333333333333333, 'p': 0.04, 'f': 0.05405404967129327}}]
<prevQ> مرحبا <prevA> أهلاً في مؤسسة اكسبو شوب! ما الذي يمكنني أن أفعله من أجلك؟ <Q> كم سعر شاشات التلفار <bot>:  ؟ ؟ ؟ ؟ ؟!!!!! ؟ ؟ ؟ ؟ هل يمكن أن يكون هناك شاشات في العالم ؟ ؟ ؟ ؟<|endoftext|>أهلا وسهلا بك زائرنا الكريم, أنت لم تقم بتسجيل الدخول بعد! يشرفنا أن تقوم بالدخول أو التسجيل إذا رغبت بالمشاركة
[{'rouge-1': {'r': 0.9166666666666666, 'p': 0.23404255319148937, 'f': 0.37288135269175526}, 'rouge-2': {'r': 0.9090909090909091, 'p': 0.18867924528301888, 'f': 0.3124999971533204}, 'rouge-l': {'r': 0.9166666666666666, 'p': 0.23404255319148937, 'f': 0.37288135269175526}}]
<prevQ> كم سعر شاشات التلفار <